In [ ]:
pip install -U sentence-transformers

In [2]:
# Packages

#pip install -U sentence-transformers
from termcolor import colored
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
import string
import regex as re
import zipfile
import gc
from scipy.stats import boxcox
import sys 
from collections import Counter 
from tqdm import tqdm 
import pandas as pd 
import numpy as np 
import warnings, math
from termcolor import colored
import pickle
import string
from sklearn.metrics.pairwise import cosine_similarity

# for eval
from sklearn.model_selection import train_test_split
import random


# for SVD
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
pd.set_option('display.float_format','{:.5f}'.format)

# for EMB
from sentence_transformers import SentenceTransformer, util
import scipy
from sklearn import preprocessing 

# for TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
from sklearn import preprocessing 

# for creating a network
from sklearn.cluster import KMeans 

In [13]:
EXT_FEATURES_PATH = 'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\ext_features_agg.csv'
NEW_DF_PATH = 'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\Oct_28_df.csv'
PROJECTS_DF_PATH = "D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\projects.csv"


ext_features = pd.read_csv(EXT_FEATURES_PATH)
df = pd.read_csv(NEW_DF_PATH)
projects_df = pd.read_csv(PROJECTS_DF_PATH)


In [14]:
df_train, df_test = train_test_split(df, test_size=0.1, random_state=42)

print('# donations in Train set: ', f"{len(df_train):,}")
print('# donations in Test set: ',  f"{len(df_test):,}")

print('# Donors in Train set: ', f"{len(df_train['Donor ID'].unique()):,}")
print('# Donors in Test set: ',  f"{len(df_test['Donor ID'].unique()):,}")


print('# Donors in both Train and Test sets - the ones we choose for evaluation: ',
      colored(f"{len(df_train[df_train['Donor ID'].isin(df_test['Donor ID'].values.tolist())]['Donor ID'].unique()):,}", 'blue'))



df_train = df_train[df_train['Donor ID'].isin(df_test['Donor ID'].values.tolist())].reset_index(drop=True)
df_test = df_test[df_test['Donor ID'].isin(df_train['Donor ID'].values.tolist())].reset_index(drop=True)

# sum of donation in a grouped by donor id dataset
df_main_donor_index = df.groupby(by = ['Donor ID', 'Project ID']).sum()[['Donation Amount']].reset_index().set_index('Donor ID')
df_train_donor_index = df_train.groupby(by = ['Donor ID', 'Project ID']).sum()[['Donation Amount']].reset_index().set_index('Donor ID')
df_test_donor_index = df_test.groupby(by = ['Donor ID', 'Project ID']).sum()[['Donation Amount']].reset_index().set_index('Donor ID')

# donations in Train set:  43,604
# donations in Test set:  4,845
# Donors in Train set:  790
# Donors in Test set:  617
# Donors in both Train and Test sets - the ones we choose for evaluation:  617


### Create Embeddings

In [15]:
projects_df = projects_df[projects_df['Project ID'].isin(df['Project ID'])].reset_index(drop=True)

project_txt = projects_df.loc[:, 'project_txt']
projects_id = projects_df['Project ID'].tolist()


In [ ]:
# ----------------- embeddings---------------------#
model = SentenceTransformer('paraphrase-distilroberta-base-v1') # , device= 'cuda'
embeddings = model.encode(project_txt, batch_size=256, show_progress_bar=True)

In [ ]:
with open(EMBEDDING_PATH, 'wb') as f:
    pickle.dump(embeddings, f)

### Donor Profile (donor embeddings & donor df)

In [5]:
EMBEDDING_PATH = 'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\Oct_28_project_embeddings.pickle'
project_embeddings = pd.read_pickle(EMBEDDING_PATH)

In [16]:
# ==================== Embeddings ==================== #

# ----------------------- projects profiles------------------- #
def get_project_profile_emb(project_id: str, project_embeddings):

    # get the ids
    idx = projects_id.index(project_id)
    project_profile = project_embeddings[idx:idx+1]
    
    return project_profile



def get_projects_profiles_emb(ids: pd.Series, project_embeddings):


    profiles_list = [get_project_profile_emb(project_id, project_embeddings)[0] for project_id in np.ravel([ids])]
    project_profiles = np.vstack(profiles_list)

    return project_profiles


# ----------------------- Donors profiles------------------- #

def build_donors_profile_emb(donor_id: str, df_train_donor_index: pd.DataFrame):

    # get the id of each person and the projects they
    # donated to
    donations_donor_df = df_train_donor_index.loc[donor_id]


    # get the vectors of projects this person has donated to
    donor_donated_project_intrain_profiles = get_projects_profiles_emb(donations_donor_df['Project ID'], project_embeddings)


    # get the smoothed donated amount as the weight of each project
    donor_project_strengths = np.array(donations_donor_df['Donation Amount']).reshape(-1, 1)


    # multiply the weights and tfidf vectors
    multiplication = np.multiply(donor_donated_project_intrain_profiles, donor_project_strengths)


    # now we normalize the whole vector 
    normalized_donor_preference = preprocessing.normalize(np.sum(multiplication, axis=0).reshape(1, -1))
    


    return normalized_donor_preference


def build_donors_profiles_emb(df_test_donor_index: pd.DataFrame, df_train_donor_index: pd.DataFrame, sample_size: int = None):
    
    # now for all donors we build a profile in a dictionary
    donor_profiles = {}
    donors_in_test_set = df_test_donor_index.index.unique().tolist()[:sample_size]

    for donor_id in tqdm(donors_in_test_set[:sample_size], position=0, leave=True):
        donor_profiles[donor_id] = build_donors_profile_emb(donor_id, df_train_donor_index)

    return donor_profiles

    



In [17]:
donor_profiles_emb = build_donors_profiles_emb(df_test_donor_index, df_train_donor_index)

100%|██████████| 617/617 [00:00<00:00, 1377.75it/s]


In [ ]:
len(donor_profiles_emb['009d5fc7b87883ffad248db5150bf1fc'][0]) == 768

In [ ]:
PROFILE_EMBEDDING_PATH = 'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\Oct_28_donor_profiles_embeddings.pickle'

with open(PROFILE_EMBEDDING_PATH, 'wb') as f:
    pickle.dump(donor_profiles_emb, f)

In [28]:
# Donor df----------------------------------------------------------

df_ = pd.merge(df, ext_features[['zip_first_three_digits', 'kmeans_cluster']],
             on='zip_first_three_digits', how='inner')

user_profile_df = df_.groupby('Donor ID'). agg(
    {
    'zip_first_three_digits': 'first',
     'kmeans_cluster': 'first',
     'Donation Amount' : ['sum', 'median'],
     'Project Cost': 'median', 
     'Project Subject Category Tree' : lambda x: ", ".join(x), 
     'Project ID' : lambda x: ",".join(x), 
     'School Metro Type' : lambda x: ",".join(x),
     'Project Grade Level Category' : lambda x:",".join(x),
     'Donation ID' : 'count'

    }
).reset_index()

In [29]:
def organize_features(text):
    cleaned_features = [i.strip() for i in text.split(",")]
    return dict(Counter(cleaned_features))

user_profile_df['subject_categories'] = user_profile_df['Project Subject Category Tree']['<lambda>'].apply(organize_features)
user_profile_df['grades'] = user_profile_df['Project Grade Level Category']['<lambda>'].apply(organize_features)
user_profile_df['metro_types'] = user_profile_df['School Metro Type']['<lambda>'].apply(organize_features)
user_profile_df['donated_projects'] = user_profile_df['Project ID']['<lambda>']
user_profile_df['donor_zip_first_three_digits'] = user_profile_df['zip_first_three_digits']['first']
user_profile_df['donor_kmeans_cluster'] = user_profile_df['kmeans_cluster']['first']


user_profile_df = user_profile_df.drop(['Project Grade Level Category',
                                  'Project Subject Category Tree', 
                                  'School Metro Type', 'kmeans_cluster',
                                  'Donation ID', 'zip_first_three_digits',
                                  'Project ID'], 
                                 axis=1)

user_profile_df.head(2)

C:\Users\Lenovo\anaconda3\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


Donor ID Donation Amount         Project Cost  \
                                                sum  median       median   
0  009d5fc7b87883ffad248db5150bf1fc       200.95590 0.44020    591.50000   
1  011ca63acc31287a150a693d92235160        36.03670 0.44020    451.20000   

                                  subject_categories  \
                                                       
0  {'Literacy & Language': 189, 'Music & The Arts...   
1  {'Literacy & Language': 24, 'Music & The Arts'...   

                                              grades  \
                                                       
0  {'Grades 9-12': 17, 'Grades PreK-2': 195, 'Gra...   
1  {'Grades PreK-2': 39, 'Grades 6-8': 22, 'Grade...   

                       metro_types  \
                                     
0  {'urban': 288, 'suburban': 105}   
1    {'urban': 40, 'suburban': 35}   

                                    donated_projects  \
                                                       
0  004a152bbe8952ea5e9d5ef89c179933,004a152bbe895...   
1  010732a68a6a6a7a40b6827355bd2a04,02180b9e9229c...   

  donor_zip_first_three_digits donor_kmeans_cluster  
                                                     
0                          782                    1  
1                          295                    0

In [31]:
USER_PROFILE_PATH = 'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\Oct_28_user_profile.csv'
user_profile_df.to_csv(USER_PROFILE_PATH, index=False)

### Graph

In [34]:
PROFILE_EMBEDDING_PATH = r'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\Oct_28_donor_profiles_embeddings.pickle'
PROJECT_EMBEDDING_PATH = r'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\Oct_28_project_embeddings.pickle'
USER_PROFILE_PATH = r'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\Oct_28_user_profile.csv'

project_embeddings = pd.read_pickle(PROJECT_EMBEDDING_PATH)
donor_profiles_emb = pd.read_pickle(PROFILE_EMBEDDING_PATH)
user_profile_df = pd.read_csv(USER_PROFILE_PATH)